In [3]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings("ignore")

In [33]:
try:
    books = pd.read_csv(r"C:\Users\adity\Book_Recommendation_System\data\Books (1).csv")
    users = pd.read_csv(r"C:\Users\adity\Book_Recommendation_System\data\Users.csv", encoding="latin1")
    ratings=pd.read_csv(r"C:\Users\adity\Book_Recommendation_System\data\Ratings (1).csv", encoding="latin1")
    print("All datasets loaded successfully")

except FileNotFoundError as e:
    print("File Not Found", e)

except pd.errors.EmptyDataError:
    print("One of the data is empty")

except pd.errors.ParserError:
    print("Error while parsing CSV file")

except Exception as e:
    print("Unexpected Error Occured",e)

All datasets loaded successfully


In [35]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [37]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [39]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [41]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [47]:
books.shape

(271360, 8)

In [53]:
books=books.dropna()

In [79]:
books.duplicated().sum()

0

In [55]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
Image-URL-S            0
Image-URL-M            0
Image-URL-L            0
dtype: int64

In [57]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [81]:
users.duplicated().sum()

0

In [61]:
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [63]:
users.shape

(278858, 3)

In [65]:
users=users.dropna()

In [67]:
users.isnull().sum()

User-ID     0
Location    0
Age         0
dtype: int64

In [71]:
users["Age"].value_counts()

Age
24.0     5687
25.0     5618
26.0     5547
23.0     5456
27.0     5383
         ... 
168.0       1
132.0       1
159.0       1
186.0       1
127.0       1
Name: count, Length: 165, dtype: int64

In [73]:
users.info()

<class 'pandas.core.frame.DataFrame'>
Index: 168096 entries, 1 to 278854
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   168096 non-null  int64  
 1   Location  168096 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 5.1+ MB


In [75]:
users.head()

,User-ID,Location,Age
1,2,"stockton, california, usa",18.0
3,4,"porto, v.n.gaia, portugal",17.0
5,6,"santa monica, california, usa",61.0
9,10,"albacete, wisconsin, spain",26.0
10,11,"melbourne, victoria, australia",14.0


In [83]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [85]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [87]:
ratings.duplicated().sum()

0

In [89]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [95]:
print("Books Columns :",books.columns)
print("Users Columns :",users.columns)
print("Ratings Columns :",ratings.columns)

Books Columns : Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')
Users Columns : Index(['User-ID', 'Location', 'Age'], dtype='object')
Ratings Columns : Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')


## Merging Books, Users, Ratings data

In [99]:
df=ratings.merge(books, on="ISBN")

In [101]:
df=df.merge(users, on="User-ID")

In [103]:
df

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Location,Age
0,276727,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,"h, new south wales, australia",16.0
1,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...,"rijeka, n/a, croatia",16.0
2,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...,"rijeka, n/a, croatia",16.0
3,276733,2080674722,0,Les Particules Elementaires,Michel Houellebecq,1998,Flammarion,http://images.amazon.com/images/P/2080674722.0...,http://images.amazon.com/images/P/2080674722.0...,http://images.amazon.com/images/P/2080674722.0...,"paris, n/a, france",37.0
4,276747,0060517794,9,Little Altars Everywhere,Rebecca Wells,2003,HarperTorch,http://images.amazon.com/images/P/0060517794.0...,http://images.amazon.com/images/P/0060517794.0...,http://images.amazon.com/images/P/0060517794.0...,"iowa city, iowa, usa",25.0
...,...,...,...,...,...,...,...,...,...,...,...,...
753290,276690,0590581066,0,Triplet Trouble and the Red Heart Race (Triple...,Debbie Dadey,1996,Scholastic Paperbacks,http://images.amazon.com/images/P/0590581066.0...,http://images.amazon.com/images/P/0590581066.0...,http://images.amazon.com/images/P/0590581066.0...,"wakeman, ohio, usa",43.0
753291,276690,0590907301,0,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey,1997,Apple,http://images.amazon.com/images/P/0590907301.0...,http://images.amazon.com/images/P/0590907301.0...,http://images.amazon.com/images/P/0590907301.0...,"wakeman, ohio, usa",43.0
753292,276706,0679447156,0,Eight Weeks to Optimum Health: A Proven Progra...,Andrew Weil,1997,Alfred A. Knopf,http://images.amazon.com/images/P/0679447156.0...,http://images.amazon.com/images/P/0679447156.0...,http://images.amazon.com/images/P/0679447156.0...,"quebec, quebec, canada",18.0
753293,276709,0515107662,10,The Sherbrooke Bride (Bride Trilogy (Paperback)),Catherine Coulter,1996,Jove Books,http://images.amazon.com/images/P/0515107662.0...,http://images.amazon.com/images/P/0515107662.0...,http://images.amazon.com/images/P/0515107662.0...,"mannington, west virginia, usa",38.0


In [106]:
df.columns

Index(['User-ID', 'ISBN', 'Book-Rating', 'Book-Title', 'Book-Author',
       'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M',
       'Image-URL-L', 'Location', 'Age'],
      dtype='object')

In [108]:
df.isnull().sum()

User-ID                0
ISBN                   0
Book-Rating            0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
Image-URL-S            0
Image-URL-M            0
Image-URL-L            0
Location               0
Age                    0
dtype: int64

In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753295 entries, 0 to 753294
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   User-ID              753295 non-null  int64  
 1   ISBN                 753295 non-null  object 
 2   Book-Rating          753295 non-null  int64  
 3   Book-Title           753295 non-null  object 
 4   Book-Author          753295 non-null  object 
 5   Year-Of-Publication  753295 non-null  object 
 6   Publisher            753295 non-null  object 
 7   Image-URL-S          753295 non-null  object 
 8   Image-URL-M          753295 non-null  object 
 9   Image-URL-L          753295 non-null  object 
 10  Location             753295 non-null  object 
 11  Age                  753295 non-null  float64
dtypes: float64(1), int64(2), object(9)
memory usage: 69.0+ MB


In [112]:
df.describe()

,User-ID,Book-Rating,Age
count,753295.000000,753295.000000,753295.000000
mean,141231.310485,2.768886,37.397669
std,78828.596534,3.864225,14.098263
min,2.000000,0.000000,0.000000
25%,74453.000000,0.000000,28.000000
50%,141493.000000,0.000000,35.000000
75%,209156.000000,7.000000,45.000000
max,278852.000000,10.000000,244.000000


In [114]:
df["Book-Rating"].value_counts()

Book-Rating
0     483679
8      64823
10     53942
7      45538
9      45249
5      27744
6      21445
4       5096
3       3331
2       1562
1        886
Name: count, dtype: int64

In [116]:
df.shape

(753295, 12)

In [134]:
rating_counts = df["Book-Rating"].value_counts()
total_ratings = rating_counts.sum()
rating_percentage = (rating_counts/total_ratings) * 100

print("Total Ratings : ",total_ratings)
print("Percentage of Ratings :",rating_percentage)

Total Ratings :  753295
Percentage of Ratings : Book-Rating
0     64.208444
8      8.605261
10     7.160807
7      6.045175
9      6.006810
5      3.683019
6      2.846826
4      0.676495
3      0.442191
2      0.207356
1      0.117617
Name: count, dtype: float64


In [124]:
ratings_per_user = df.groupby("User-ID")["Book-Rating"].count()

In [140]:
ratings_per_user

User-ID
2          1
10         1
19         1
20         1
36         1
          ..
278844     2
278846     1
278849     4
278851    23
278852     1
Name: Book-Rating, Length: 53887, dtype: int64

In [142]:
ratings_per_user.describe()

count    53887.000000
mean        13.979160
std         91.545745
min          1.000000
25%          1.000000
50%          1.000000
75%          5.000000
max       6456.000000
Name: Book-Rating, dtype: float64

In [146]:
books.nunique()

ISBN                   271353
Book-Title             242129
Book-Author            102019
Year-Of-Publication       200
Publisher               16803
Image-URL-S            271037
Image-URL-M            271037
Image-URL-L            271037
dtype: int64

In [ ]:
ratings_per_book=df.groupby("Book-Title ")["Book-Rating"].count()